# Question Answering

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023. LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [ ]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
print(vectordb._collection.count())

In [ ]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

## RetrievalQA chain
<p align = "center">
    <img src="./image1.png">
</p>

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]

## Prompt
What going on underneeth the hood

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """
    Use the following pieces of context to answer th equestion at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
    {context}
    Question: {question}
    Helpful Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.template(
    template=template
)

In [ ]:
# Run chain
qa_chain = RetrievalQA.chain_from_type(
    llm,
    retriever = vectordb.as_retriever(),
    return_source_documents = True,
    chain_type_kwargs = {"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Is probability a class topic?"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]

In [ ]:
result["source_documents"][0]

this was using the `stuff` technique in which we just stuff the context into the prompt, below as some other methods discussed

- good because requires only one call to the LLM model
- bad becausse sometimes  one call may not fit all or a good amount of context

## RetrievalQA chain types
<p align = "center">
    <img src = "./image2.png">
</p>

In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [ ]:
result = qa_chain_mr({"query": question})

In [ ]:
result["result"]

## RetrievalQA limitations
QA fails to preserve conversational history.